<a href="https://colab.research.google.com/github/Pheonix10101/Final_project_prcv/blob/main/Tranfer_learining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Final Project:Pneumonia Detection from Chest X-ray: A CNN Aproach**

Author:
Samruddhi Raut.
This file contain the Transfer learning models.





**Import Statements**

In [ ]:
import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras



**Reading through drive**

In [ ]:
os.listdir("/content/drive/MyDrive/archive(1)/chest_xray")

['chest_xray', 'test', 'train', '__MACOSX', 'val']

**Total no of Pnemonia x-ray**

In [ ]:
len(os.listdir("/content/drive/MyDrive/archive(1)/chest_xray/chest_xray/train/PNEUMONIA"))

3876

In [ ]:
train_dir = "/content/drive/MyDrive/archive(1)/chest_xray/chest_xray/train"
test_dir = "/content/drive/MyDrive/archive(1)/chest_xray/chest_xray/test"
val_dir = "/content/drive/MyDrive/archive(1)/chest_xray/chest_xray/val"

print("Train set:\n========================================")
num_pneumonia = len(os.listdir(os.path.join(train_dir, 'PNEUMONIA')))
num_normal = len(os.listdir(os.path.join(train_dir, 'NORMAL')))
print(f"PNEUMONIA={num_pneumonia}")
print(f"NORMAL={num_normal}")

print("Test set:\n========================================")
print(f"PNEUMONIA={len(os.listdir(os.path.join(test_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(test_dir, 'NORMAL')))}")

print("Validation set:\n========================================")
print(f"PNEUMONIA={len(os.listdir(os.path.join(val_dir, 'PNEUMONIA')))}")
print(f"NORMAL={len(os.listdir(os.path.join(val_dir, 'NORMAL')))}")


Train set:
PNEUMONIA=3876
NORMAL=1342
Test set:
PNEUMONIA=390
NORMAL=234
Validation set:
PNEUMONIA=9
NORMAL=9


**Image Processing**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

image_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True
)


In [ ]:
train = image_generator.flow_from_directory(train_dir, 
                                            batch_size=8, 
                                            shuffle=True, 
                                            class_mode='binary',
                                            target_size=(180, 180))

validation = image_generator.flow_from_directory(val_dir, 
                                                batch_size=1, 
                                                shuffle=False, 
                                                class_mode='binary',
                                                target_size=(180, 180))

test = image_generator.flow_from_directory(test_dir, 
                                            batch_size=1, 
                                            shuffle=False, 
                                            class_mode='binary',
                                            target_size=(180, 180))

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


**Weight for classes**

In [ ]:
weight_for_0 = num_pneumonia / (num_normal + num_pneumonia)
weight_for_1 = num_normal / (num_normal + num_pneumonia)

class_weight = {0: weight_for_0, 1: weight_for_1}

print(f"Weight for class 0: {weight_for_0:.2f}")
print(f"Weight for class 1: {weight_for_1:.2f}")

Weight for class 0: 0.74
Weight for class 1: 0.26


**Transfer learning**

**DenseNet** 121

In [ ]:
from keras.applications.densenet import DenseNet121
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras import backend as K

base_model = DenseNet121(input_shape=(180, 180, 3), include_top=False, weights='imagenet', pooling='avg')

base_model.summary()

29097984/29084464 [==============================] - 0s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 186, 186, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 90, 90, 64)   9408        ['zero_padding2d[0][0]']         
                                                                                                  
 conv1/bn (BatchNormali

In [ ]:
layers = base_model.layers
print(f"The model has {len(layers)} layers")

The model has 428 layers


In [ ]:
print(f"The input shape {base_model.input}")
print(f"The output shape {base_model.output}")

The input shape KerasTensor(type_spec=TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
The output shape KerasTensor(type_spec=TensorSpec(shape=(None, 1024), dtype=tf.float32, name=None), name='avg_pool/Mean:0', description="created by layer 'avg_pool'")


**Defining Loss and Activation Function**

In [ ]:

base_model = DenseNet121(include_top=False, weights='imagenet')
x = base_model.output

x = GlobalAveragePooling2D()(x)

predictions = Dense(1, activation="sigmoid")(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [ ]:


r = model.fit(
    train, 
    epochs=10,
    validation_data=validation,
    class_weight=class_weight,
    steps_per_epoch=50,
    validation_steps=25,
)



Epoch 1/10
50/50 [==============================] - 85s 1s/step - loss: 0.1774 - accuracy: 0.8350 - val_loss: 5.6201 - val_accuracy: 0.5000
Epoch 2/10
50/50 [==============================] - 63s 1s/step - loss: 0.1483 - accuracy: 0.8275
Epoch 3/10
50/50 [==============================] - 61s 1s/step - loss: 0.0917 - accuracy: 0.9100
Epoch 4/10
50/50 [==============================] - 57s 1s/step - loss: 0.0961 - accuracy: 0.8950
Epoch 5/10
50/50 [==============================] - 54s 1s/step - loss: 0.0619 - accuracy: 0.9325
Epoch 6/10
50/50 [==============================] - 49s 970ms/step - loss: 0.0907 - accuracy: 0.9050
Epoch 7/10
50/50 [==============================] - 58s 1s/step - loss: 0.0630 - accuracy: 0.9300
Epoch 8/10
50/50 [==============================] - 47s 936ms/step - loss: 0.1032 - accuracy: 0.8850
Epoch 9/10
50/50 [==============================] - 41s 819ms/step - loss: 0.0719 - accuracy: 0.9200
Epoch 10/10
50/50 [==============================] - 47s 938ms/step

**Loss and Accracy Plot**

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evalution')

plt.subplot(2, 2, 2)
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evalution')

In [57]:
predicted_vals = model.predict(test, steps=len(test))

In [58]:

from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(test.classes, predicted_vals > 0.5))
pd.DataFrame(classification_report(test.classes, predicted_vals > 0.5, output_dict=True))


[[226   8]
 [143 247]]


,0,1,accuracy,macro avg,weighted avg
precision,0.612466,0.968627,0.758013,0.790547,0.835067
recall,0.965812,0.633333,0.758013,0.799573,0.758013
f1-score,0.749585,0.765891,0.758013,0.757738,0.759777
support,234.000000,390.000000,0.758013,624.000000,624.000000


**Evaluating Train and Test accuracy**

In [ ]:

evaluation = model.evaluate(test)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = model.evaluate(train)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")


624/624 [==============================] - 175s 280ms/step - loss: 0.6305 - accuracy: 0.7821
Test Accuracy: 78.21%
652/652 [==============================] - 480s 738ms/step - loss: 0.8860 - accuracy: 0.7166
Train Accuracy: 71.66%


**VGG16**

In [ ]:
import keras
from keras import applications
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D
from keras.applications.vgg16 import VGG16

vgg16_base_model = VGG16(input_shape=(180,180,3),include_top=False,weights='imagenet')


58900480/58889256 [==============================] - 1s 0us/step


**Model Summary**

In [ ]:
vgg16_base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [ ]:
vgg16_model = tf.keras.Sequential([
        vgg16_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64,activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1,activation="sigmoid")
    ])

**Defining Optimizer and loss function**

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
vgg16_model.compile(optimizer=opt,loss='binary_crossentropy',metrics=METRICS)

**Training**

In [ ]:
r = vgg16_model.fit(train,
          epochs=10,
          validation_data=validation,
          class_weight=class_weight,
          steps_per_epoch=100,
          validation_steps=25)

Epoch 1/10
100/100 [==============================] - 46s 395ms/step - loss: 0.2742 - accuracy: 0.6550 - precision: 0.8597 - recall: 0.6397 - val_loss: 30.7882 - val_accuracy: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
100/100 [==============================] - 26s 255ms/step - loss: 0.2359 - accuracy: 0.7013 - precision: 0.9077 - recall: 0.6770
Epoch 3/10
100/100 [==============================] - 27s 265ms/step - loss: 0.1949 - accuracy: 0.7563 - precision: 0.9143 - recall: 0.7337
Epoch 4/10
100/100 [==============================] - 26s 256ms/step - loss: 0.1844 - accuracy: 0.7800 - precision: 0.9393 - recall: 0.7534
Epoch 5/10
100/100 [==============================] - 26s 257ms/step - loss: 0.1589 - accuracy: 0.8200 - precision: 0.9535 - recall: 0.8039
Epoch 6/10
100/100 [==============================] - 26s 261ms/step - loss: 0.2001 - accuracy: 0.7887 - precision: 0.9111 - recall: 0.7880
Epoch 7/10
100/100 [==============================] - 26s 255ms/

**Loss and Accuracy Plot**

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

**Model Evaluation**

In [ ]:
evaluation =vgg16_model.evaluate(test)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = vgg16_model.evaluate(train)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

624/624 [==============================] - 23s 37ms/step - loss: 0.6016 - accuracy: 0.7917 - precision: 0.9167 - recall: 0.7333
Test Accuracy: 79.17%
652/652 [==============================] - 140s 214ms/step - loss: 0.6713 - accuracy: 0.7650 - precision: 0.9907 - recall: 0.6901
Train Accuracy: 76.50%


**Confusion Matrix**

In [ ]:


pred_vgg = model.predict(test)

print(confusion_matrix(test.classes, pred_vgg > 0.5))
pd.DataFrame(classification_report(test.classes, pred_vgg > 0.5, output_dict=True))

[[227   7]
 [125 265]]


,0,1,accuracy,macro avg,weighted avg
precision,0.644886,0.974265,0.788462,0.809576,0.850748
recall,0.970085,0.679487,0.788462,0.824786,0.788462
f1-score,0.774744,0.800604,0.788462,0.787674,0.790907
support,234.000000,390.000000,0.788462,624.000000,624.000000


**ResNet50**

In [ ]:

from tensorflow.keras.applications.resnet50 import ResNet50

resnet_base_model = ResNet50(input_shape=(180,180,3), include_top=False, weights='imagenet')



94781440/94765736 [==============================] - 1s 0us/step


**Model Summary**

In [ ]:
resnet_base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 180, 180, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 186, 186, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 90, 90, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 90, 90, 64)   256         ['conv1_conv[0][0]']      

**Defining activation function**

In [ ]:
resnet_model = tf.keras.Sequential([
        resnet_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64,activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1,activation="sigmoid")
    ])

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
resnet_model.compile(optimizer=opt,loss='binary_crossentropy',metrics=METRICS)

**Training**

In [ ]:
r = resnet_model.fit(train,
          epochs=10,
          validation_data=validation,
          class_weight=class_weight,
          steps_per_epoch=100,
          validation_steps=25)

Epoch 1/10
100/100 [==============================] - 41s 306ms/step - loss: 0.3040 - accuracy: 0.6012 - precision: 0.8407 - recall: 0.5745 - val_loss: 0.8990 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/10
100/100 [==============================] - 43s 427ms/step - loss: 0.2600 - accuracy: 0.6413 - precision: 0.8456 - recall: 0.6342
Epoch 3/10
100/100 [==============================] - 29s 287ms/step - loss: 0.2291 - accuracy: 0.7225 - precision: 0.9022 - recall: 0.7010
Epoch 4/10
100/100 [==============================] - 29s 291ms/step - loss: 0.1663 - accuracy: 0.8050 - precision: 0.9531 - recall: 0.7783
Epoch 5/10
100/100 [==============================] - 29s 292ms/step - loss: 0.1699 - accuracy: 0.8175 - precision: 0.9411 - recall: 0.8050
Epoch 6/10
100/100 [==============================] - 30s 300ms/step - loss: 0.1623 - accuracy: 0.8537 - precision: 0.9421 - recall: 0.8597
Epoch 7/10
100/100 [==============================] - 30s 293ms/step - lo

**Loss and Accuracy Plot**

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

**Model Evaluation**

In [ ]:
evaluation =resnet_model.evaluate(test)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = resnet_model.evaluate(train)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

624/624 [==============================] - 26s 42ms/step - loss: 0.7482 - accuracy: 0.6410 - precision: 0.8458 - recall: 0.5205
Test Accuracy: 64.10%
652/652 [==============================] - 172s 263ms/step - loss: 0.8322 - accuracy: 0.6219 - precision: 0.9887 - recall: 0.4968
Train Accuracy: 62.19%


**Result Prediction and confusion Matrix**

In [ ]:
pred_res50 = model.predict(test)

print(confusion_matrix(test.classes, pred_res50 > 0.5))
pd.DataFrame(classification_report(test.classes, pred_res50 > 0.5, output_dict=True))

[[229   5]
 [129 261]]


,0,1,accuracy,macro avg,weighted avg
precision,0.639665,0.981203,0.785256,0.810434,0.853126
recall,0.978632,0.669231,0.785256,0.823932,0.785256
f1-score,0.773649,0.795732,0.785256,0.784690,0.787451
support,234.000000,390.000000,0.785256,624.000000,624.000000


**InceptionV3**

In [ ]:
from tensorflow.keras.applications import InceptionV3

inception_base_model = InceptionV3(input_shape=(180,180,3),include_top=False,weights='imagenet')

87924736/87910968 [==============================] - 1s 0us/step


**Defining activation function and optimizer**

In [ ]:
inception_model = tf.keras.Sequential([
        inception_base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.6),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),
        Dense(64,activation="relu"),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1,activation="sigmoid")
    ])

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
inception_model.compile(optimizer=opt,loss='binary_crossentropy',metrics=METRICS)

**Model Training**

In [ ]:
r = inception_model.fit(train,
          epochs=10,
          validation_data=validation,
          class_weight=class_weight,
          steps_per_epoch=100,
          validation_steps=25)

Epoch 1/10
100/100 [==============================] - 54s 417ms/step - loss: 0.2201 - accuracy: 0.7025 - precision: 0.9035 - recall: 0.6609 - val_loss: 9.7897 - val_accuracy: 0.5000 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/10
100/100 [==============================] - 25s 253ms/step - loss: 0.2050 - accuracy: 0.7425 - precision: 0.9134 - recall: 0.7298
Epoch 3/10
100/100 [==============================] - 25s 245ms/step - loss: 0.1655 - accuracy: 0.8238 - precision: 0.9382 - recall: 0.8227
Epoch 4/10
100/100 [==============================] - 26s 257ms/step - loss: 0.1665 - accuracy: 0.8150 - precision: 0.9293 - recall: 0.8085
Epoch 5/10
100/100 [==============================] - 26s 254ms/step - loss: 0.1697 - accuracy: 0.7925 - precision: 0.9427 - recall: 0.7696
Epoch 6/10
100/100 [==============================] - 25s 251ms/step - loss: 0.1774 - accuracy: 0.7850 - precision: 0.9354 - recall: 0.7678
Epoch 7/10
100/100 [==============================] - 25s 252ms/step - lo

**Loss and Accuracy Plot**

In [ ]:
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(r.history['loss'], label='Loss')
plt.plot(r.history['val_loss'], label='Val_Loss')
plt.legend()
plt.title('Loss Evolution')

plt.subplot(2, 2, 2)
plt.plot(r.history['accuracy'], label='Accuracy')
plt.plot(r.history['val_accuracy'], label='Val_Accuracy')
plt.legend()
plt.title('Accuracy Evolution')

**Model Evaluation**

In [ ]:
evaluation =inception_model.evaluate(test)
print(f"Test Accuracy: {evaluation[1] * 100:.2f}%")

evaluation = inception_model.evaluate(train)
print(f"Train Accuracy: {evaluation[1] * 100:.2f}%")

624/624 [==============================] - 27s 43ms/step - loss: 0.5047 - accuracy: 0.8109 - precision: 0.8523 - recall: 0.8436
Test Accuracy: 81.09%
652/652 [==============================] - 151s 232ms/step - loss: 0.6711 - accuracy: 0.8391 - precision: 0.9925 - recall: 0.7894
Train Accuracy: 83.91%


**Result Prediction and confusion matrix**

In [ ]:
pred_inc = model.predict(test)

print(confusion_matrix(test.classes, pred_inc > 0.5))
pd.DataFrame(classification_report(test.classes, pred_inc > 0.5, output_dict=True))

[[225   9]
 [131 259]]


,0,1,accuracy,macro avg,weighted avg
precision,0.632022,0.966418,0.775641,0.799220,0.841020
recall,0.961538,0.664103,0.775641,0.812821,0.775641
f1-score,0.762712,0.787234,0.775641,0.774973,0.778038
support,234.000000,390.000000,0.775641,624.000000,624.000000
